# COGS 108 - Data Checkpoint

# Names

- Zhaoyi Yu
- Zhuoran Li
- Zecheng Li
- Gao Mo

<a id='research_question'></a>
# Research Question

Is the obesity rate associated with the living condition of a countries citizens? Living condition is defined by the income level, happiness scores, and the weather conditions indicated by a country’s region in latitude and longitude.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

- Dataset Name: 2015 - 2019.csv 
- Link to the dataset: https://www.kaggle.com/unsdsn/world-happiness
- Number of observations: 158 observations per year. 790 observations in total
- Description: This set of datasets is the annual data on citizens’ living conditions in each country (ie. GDP, happiness level, Income rate, etc...) ranging from 2015 to 2019. 
From these datasets, we will be extracting the ‘happiness score’ and GDP for each country. These datasets will also be used as indexes for our dataset.



- Dataset Name: Obesity_clean.csv
- Link to the dataset: https://www.kaggle.com/amanarora/obesity-among-adults-by-country-19752016
- Number of observations: 24570 observations in total 
- Description: The obesity_clean dataset depicts the obesity rate by percentage annually divided by male and female. 
From this dataset, we will be using the obesity rate itself and the country and year information along with it to correspond to the 2015-2017 dataset. To do that, we will need to combine the male and female obesity rates to show obesity levels as a whole.



- Dataset Name: country_median dataset
- Link to the dataset: https://ourworldindata.org/grapher/daily-median-income?country=OWID_WRL~ESP~KOR~MDG
- Number of observations: 6953 observations in total 
- Description: The country income dataset contains the level of income for all countries ranging from the year 1981 to 2019. 
Note that this dataset only shows the average daily income of each income and is somewhat tricky to analyze since the ideal dataset would be income level as a whole.
We will be extracting data of the country’s average income from 2015 to 2019 for our project. 




- Dataset Name: Latitude dataset
- Link to the dataset: https://www.kaggle.com/paultimothymooney/latitude-and-longitude-for-every-country-and-state
- Number of observations: 244 observations in total 
- Description: This dataset contains the latitude/longitude data corresponding to the countries. 
We will be using the dataset as an indicator of the weather in each region by comparing it to the weather information in those areas.







- Combining: For our project specifically, we will extract the countries as indexes combined from 2015-2016 datasets and merge them with the obesity, income, and weather datasets with matching years and countries. Hence, our resulting data would country-based data with obesity, happiness, and income level align with the years from 2015 to 2016.

# Setup

In [416]:
import pandas as pd
import os 
import numpy as np
import requests
import geopandas as gpd

# Data Cleaning

# Import all the CSV files

In [493]:
# import CSV Files
happy_2015 = pd.read_csv('2015.csv')
happy_2016 = pd.read_csv('2016.csv')
happy_2017 = pd.read_csv('2017.csv')
happy_2018 = pd.read_csv('2018.csv')
happy_2019 = pd.read_csv('2019.csv')

country_income = pd.read_csv('daily-median-income.csv')

obesity = pd.read_csv('obesity-cleaned.csv')

df_pos = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')



# Daily-median-income Dataset Cleaning

In [494]:
print(' ')
print('Original dataset:')
country_income.head(3)

 
Original dataset:


,Entity,Code,Year,50th percentile (median) – level of income or consumption per day
0,Albania,ALB,1981,6.38
1,Albania,ALB,1982,6.43
2,Albania,ALB,1983,6.36


Since the entries of dataset from every country are separated by year, it is then reasonable to group the dataset by each entity (country) and then calculate the average amount of income per day. For the merging process later, we change the name of the entity column to 'Country'. Since the value for income per year per county is daily, we multiplied that number by 365 to compute the average yearly median income. Change the name of the income column to median_income. The reason why we only included the entries from 2015 and 16 is that the obesity dataset contains the data point only from those two years. 

In [495]:
clean_income_15 = country_income[country_income.Year == 2015].groupby('Entity')\
                                           ['50th percentile (median) – level of income or consumption per day']\
                                            .agg('mean').to_frame().reset_index().rename(columns = {'Entity':'Country'})
clean_income_15['50th percentile (median) – level of income or consumption per day']  = \
                            clean_income_15['50th percentile (median) – level of income or consumption per day'].apply(lambda x: x*365)
clean_income_15 = clean_income_15.rename(columns = {'50th percentile (median) – level of income or consumption per day':'median_income_2015'})
print(' ')
print('Cleaned dataset for 2015:')
clean_income_15.head(3)




 
Cleaned dataset for 2015:


,Country,median_income_2015
0,Albania,2544.05
1,Algeria,2868.90
2,Angola,868.70


In [496]:
clean_income_16 = country_income[country_income.Year == 2016].groupby('Entity')\
                                           ['50th percentile (median) – level of income or consumption per day']\
                                            .agg('mean').to_frame().reset_index().rename(columns = {'Entity':'Country'})
clean_income_16['50th percentile (median) – level of income or consumption per day']  = \
                            clean_income_16['50th percentile (median) – level of income or consumption per day'].apply(lambda x: x*365)
clean_income_16 = clean_income_16.rename(columns = {'50th percentile (median) – level of income or consumption per day':'median_income_2016'})
print(' ')
print('Cleaned dataset for 2016:')
clean_income_16.head(3)




 
Cleaned dataset for 2016:


,Country,median_income_2016
0,Albania,2598.8
1,Algeria,2905.4
2,Angola,824.9


# Yearly Happiness Score Dataset Cleaning

In [497]:
print(' ')
print('Original dataset:')
happy_2015.head(3)

 
Original dataset:


,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204


By analysing the given dataset, we noticed that the happiness score is the sum of all the other columns,hence we dropped the rest of the columns other than Country, Happiness rank, as well as the Happiness score. For the merging process later, we changed the names designated to each country. 

In [498]:
clean_2015 = happy_2015[['Country','Happiness Rank','Happiness Score']] \
                    .rename(columns = {'Happiness Rank':'Happiness_Rank_15','Happiness Score':'Happiness_Score_15'})

clean_2016 = happy_2016[['Country','Happiness Rank','Happiness Score']] \
                .rename(columns ={'Happiness Rank':'Happiness_Rank_16','Happiness Score':'Happiness_Score_16'})

clean_2017 = happy_2017[['Country','Happiness.Rank','Happiness.Score']] \
                .rename(columns = {'Happiness.Rank':'Happiness_Rank_17','Happiness.Score':'Happiness_Score_17' } )

clean_2018 = happy_2018[['Country or region','Overall rank','Score']] \
                .rename(columns = {'Country or region':'Country', 'Overall rank':'Happiness_Rank_18', 'Score':'Happiness_Score_18'})

clean_2019 = happy_2019[['Country or region','Overall rank','Score']] \
                .rename(columns = {'Country or region':'Country', 'Overall rank':'Happiness_Rank_19', 'Score':'Happiness_Score_19'})
print(' ')
print('Cleaned dataset:')
clean_2015.head(3)


 
Cleaned dataset:


,Country,Happiness_Rank_15,Happiness_Score_15
0,Switzerland,1,7.587
1,Iceland,2,7.561
2,Denmark,3,7.527


In [499]:
clean_2016.head(3)

,Country,Happiness_Rank_16,Happiness_Score_16
0,Denmark,1,7.526
1,Switzerland,2,7.509
2,Iceland,3,7.501


In [500]:
clean_2017.head(3)

,Country,Happiness_Rank_17,Happiness_Score_17
0,Norway,1,7.537
1,Denmark,2,7.522
2,Iceland,3,7.504


In [501]:
clean_2018.head(3)

,Country,Happiness_Rank_18,Happiness_Score_18
0,Finland,1,7.632
1,Norway,2,7.594
2,Denmark,3,7.555


In [502]:
clean_2019.head(3)

,Country,Happiness_Rank_19,Happiness_Score_19
0,Finland,1,7.769
1,Denmark,2,7.600
2,Norway,3,7.554


# Country Latitude Data Cleaning

In [503]:
print(' ')
print('Original dataset:')
df_pos.head(3)

 
Original dataset:


,country_code,latitude,longitude,country,usa_state_code,usa_state_latitude,usa_state_longitude,usa_state
0,AD,42.546245,1.601554,Andorra,AK,63.588753,-154.493062,Alaska
1,AE,23.424076,53.847818,United Arab Emirates,AL,32.318231,-86.902298,Alabama
2,AF,33.939110,67.709953,Afghanistan,AR,35.201050,-91.831833,Arkansas


A part of our hypothesis is that the weather and temperatue in one's living area could effect their diet condition as well as the motivation to exercise outisde. Hence, we categoried the countries into 8 regions based on their location on earth (i.e. Latitude). Again, for the later merging process, we changed the name of the columns

In [504]:
df_pos = df_pos[['country','latitude']]

def area(lat):
    if abs(lat) <= 10:
        return 1
    elif abs(lat) <= 20:
        return 2
    elif abs(lat) <= 30:
        return 3
    elif abs(lat) <= 40:
        return 4
    elif abs(lat) <= 50:
        return 5
    elif abs(lat) <= 60:
        return 6
    elif abs(lat) <= 70:
        return 7
    elif abs(lat) <= 80:
        return 8
    elif abs(lat) <= 90:
        return 9    
    
df_pos['area'] = df_pos['latitude'].apply(area)
df_pos = df_pos.drop(columns='latitude')
df_pos = df_pos.dropna()
df_pos = df_pos.rename(columns = {'country':'Country'})
print(' ')
print('Cleaned dataset:')
df_pos.head(3)

 
Cleaned dataset:


,Country,area
0,Andorra,5.0
1,United Arab Emirates,3.0
2,Afghanistan,4.0


In [505]:
obesity = pd.read_csv('obesity-cleaned.csv')
obesity.dropna()

,Unnamed: 0,Country,Year,Obesity (%),Sex
0,0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes
1,1,Afghanistan,1975,0.2 [0.0-0.6],Male
2,2,Afghanistan,1975,0.8 [0.2-2.0],Female
3,3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes
4,4,Afghanistan,1976,0.2 [0.0-0.7],Male
...,...,...,...,...,...
24565,24565,Zimbabwe,2015,4.5 [2.4-7.6],Male
24566,24566,Zimbabwe,2015,24.8 [18.9-31.3],Female
24567,24567,Zimbabwe,2016,15.5 [12.0-19.2],Both sexes
24568,24568,Zimbabwe,2016,4.7 [2.5-8.0],Male


In [506]:
obesity.dropna(inplace = True)
obesity['Obesity (%)'] = obesity['Obesity (%)'].apply(lambda x: x.split('[')[0])
obesity = obesity.rename(columns = {'Obesity (%)':'obe'})
obesity = obesity[obesity.obe != 'No data']

In [507]:
obesity.obe = obesity.obe.apply(lambda x: float(x))

In [523]:
obesity_15 = obesity[obesity.Year == 2015]
obesity_16 = obesity[obesity.Year == 2016]

In [530]:
clean_obesity_15 = obesity_15.groupby('Country').obe.agg('mean').to_frame().reset_index()
clean_obesity_15 = clean_obesity_15.rename(columns = {'obe' : 'obe_15'})
clean_obesity_16 = obesity_16.groupby('Country').obe.agg('mean').to_frame().reset_index()
clean_obesity_16 = clean_obesity_16.rename(columns = {'obe' : 'obe_16'})

In [531]:
clean_obesity_15.head(3)

,Country,obe_15
0,Afghanistan,5.166667
1,Albania,21.100000
2,Algeria,26.700000


In [532]:
clean_obesity_16.head(3)

,Country,obe_16
0,Afghanistan,5.433333
1,Albania,21.700000
2,Algeria,27.400000


# Merging

In [533]:
merge1 = pd.merge(clean_2015, clean_2016, on="Country", how = 'inner')
merge2 = pd.merge(merge1, clean_2017, on="Country", how = 'inner')
merge3 = pd.merge(merge2, clean_2018, on="Country", how = 'inner')
combined_happy = pd.merge(merge3, clean_2019, on="Country", how = 'inner')
combined_income1 = pd.merge(combined_happy,clean_income_15,on="Country", how = 'inner')
combined_income = pd.merge(combined_income1,clean_income_16,on="Country", how = 'inner')
combined = pd.merge(combined_income,df_pos,on="Country", how = 'inner')
combined_ob = pd.merge(combined,clean_obesity_15,on="Country", how = 'inner')
combined_all = pd.merge(combined_ob,clean_obesity_16,on="Country", how = 'inner')

In [534]:
combined_all = combined_all.drop(['Happiness_Rank_17','Happiness_Score_17','Happiness_Rank_18','Happiness_Score_18',\
                   'Happiness_Rank_19','Happiness_Score_19'],axis = 1)

since the obesity data only contains the year 2015 and 2016, we decided to drops the columns for happiness scores and ranks from 2017-2019.

In [535]:
print(' ')
print('Final cleaned dataset:')
combined_all

 
Final cleaned dataset:


,Country,Happiness_Rank_15,Happiness_Score_15,Happiness_Rank_16,Happiness_Score_16,median_income_2015,median_income_2016,area,obe_15,obe_16
0,Switzerland,1,7.587,2,7.509,21275.85,21458.35,5.0,19.133333,19.533333
1,Iceland,2,7.561,3,7.501,17016.30,18239.05,7.0,21.466667,21.833333
2,Denmark,3,7.527,1,7.526,16709.70,17191.50,6.0,19.266667,19.666667
3,Norway,4,7.522,4,7.498,22958.50,22743.15,7.0,22.600000,23.066667
4,Canada,5,7.427,6,7.404,18067.50,18122.25,6.0,28.800000,29.400000
...,...,...,...,...,...,...,...,...,...,...
108,Burkina Faso,152,3.587,145,3.739,755.55,777.45,2.0,5.133333,5.433333
109,Rwanda,154,3.465,152,3.515,631.45,635.10,1.0,5.433333,5.666667
110,Benin,155,3.340,153,3.484,700.80,704.45,1.0,9.166667,9.500000
111,Burundi,157,2.905,157,2.905,445.30,427.05,1.0,5.100000,5.366667


### Variable Definitions

Country: Country names     
Happiness_Rank_15: Happiness Rank of each Country in 2015
Happiness_Score_15: Happiness Score of each Country in 2015	
Happiness_Rank_16: Happiness Rank of each Country in 2016
Happiness_Score_16: Happiness Score of each Country in 2016		
median_income_2015: Median Income of each Country in 2015
median_income_2016: Median Income of each Country in 2016
area: Country Region based on Latitude
obe_15: Obesity Rate of each Country in 2015
obe_16: Obesity Rate of each Country in 2016

# Checking for Null Values

In [458]:
for i in list(combined.columns):
    print(i)
    print(sum(combined[i].isna()))

Country
0
Happiness_Rank_15
0
Happiness_Score_15
0
Happiness_Rank_16
0
Happiness_Score_16
0
Happiness_Rank_17
0
Happiness_Score_17
0
Happiness_Rank_18
0
Happiness_Score_18
0
Happiness_Rank_19
0
Happiness_Score_19
0
median_income
0
area
0
